In [ ]:
'''

    ## 流程 = 語音接收-->儲存-->mfcc轉換-->判別--回傳結果 ##
    1. 音檔必須大於10s的版本
    2. 訓練資料:train_v7.json
    3. 權重:model_version_v7.h5
    
'''

In [ ]:
import json
import os
import math
import struct
import base64
import librosa
import pandas as pd
import wave
from flask import Flask, jsonify, request
from flask_cors import cross_origin
from keras.models import load_model
import speech_recognition as sr
import numpy as np
import asyncio
import speech_recognition as sr
from googletrans import Translator
import matplotlib.pyplot as plt
from gensim.models.fasttext import FastText
model = FastText.load_fasttext_format('cc.en.300.bin')

In [ ]:
# Draw
def oscillogram_spectrum():
    """
    画出音频文件audio_path的声波图和频谱图
    :param audio_path:音频文件路径
    :return:
    """
    # 读取wav文件
    filename = "./audio.wav"
    wavefile = wave.open(filename, 'r')  # open for writing
    # 读取wav文件的四种信息的函数。numframes表示一共读取了几个frames。
    nchannels = wavefile.getnchannels()
    sample_width = wavefile.getsampwidth()
    framerate = wavefile.getframerate()
    numframes = wavefile.getnframes()
    y = np.zeros(numframes)

    for index, i in enumerate(range(numframes)):
        val = wavefile.readframes(1)
        left = val[0:2]
        # right = val[2:4]
        v = struct.unpack('h', left)[0]
        # print("v", v)
        y[i] = v
    # framerate就是声音的采用率，文件初读取的值。
    Fs = framerate
    time = np.arange(0, numframes) * (1.0 / framerate)
    feature=[]
    feature_y=[]
    dict_={
        "y":y
    }

    for i, x in enumerate(time):
        if y[i]>=500 or y[i]<=-500:
            feature.append(x)
            feature_y.append(y[i])


    time_cut = round(numframes/4)
    new_time_cut = [word for word in time if word != 0.0]
    # print(new_time_cut)
    feature = np.array(feature)
    feature_y = np.array(feature_y)

#     time1 = time[ :time_cut]
#     time2 = time[time_cut : time_cut*2]
#     time3 = time[time_cut*2 : time_cut*3]
#     time4 = time[time_cut*3:]

    # 显示时域图(波形图)
    # plt.subplot(2,2,1)
    plt.subplot(1,1,1)
    a,= plt.plot(feature, feature_y, color='r')
    # plt.subplot(2,2,2)
    # plt.subplot(1,1,1)
    b, = plt.plot(time, y)

    plt.legend([a , b], ["Teacher Voice", "Your Voice"])
    # plt.subplot(1,1,1)
    # plt.subplot(2,2,1)
    # plt.plot(time1, y[ :time_cut])
    #
    # plt.subplot(2, 2, 2)
    # plt.plot(time2, y[time_cut : time_cut*2])
    #
    # plt.subplot(2, 2, 3)
    # plt.plot(time3, y[time_cut*2 : time_cut*3])
    #
    # plt.subplot(2, 2, 4)
    # plt.plot(time4, y[time_cut*3:])
    
    # 显示频域图(频谱图)
    # plt.subplot(2,2,3)
    # plt.specgram(y, NFFT=1024, Fs=Fs, noverlap=900)

    plt.subplots_adjust(hspace=0.5, wspace=0.3)
    plt.rcParams['savefig.dpi'] = 100
    plt.rcParams['figure.dpi'] = 100
    plt.ylabel('amplitude(db)',fontsize=10)
    plt.xlabel('Time(s)',fontsize=10)
    plt.savefig('./audio.png')
    plt.show()
    
def Bargraph():
    plt.plot(point_list, color="b", linewidth=3)
    plt.figsize=(12,8)
    plt.title('Mr. XXX Growth Record',fontsize=20)
    plt.xlabel('date',fontsize=12)
    plt.ylabel('point',fontsize=12)
    plt.xtick(fontsize=10)
    plt.ytick(fontsize=10)
    plt.grid(True)
    plt.show()
    
def get_wave_word(file_path):
    r = sr.Recognizer()                        #預設辨識英文
    with sr.WavFile(file_path) as source:  #讀取wav檔
        audio = r.record(source)
    try:
        return "Transcription: " + r.recognize_google(audio,language="en")
                                              #使用Google的服務
    except :
        return "請再唸一次"
    
## 11/30新增
def check_your_voice_Volume():
    normal_sound=0
    too_low_sound=0
    for index, i in enumerate(range(numframes)):
    val = wavefile.readframes(1)
    left = val[0:2]
    # right = val[2:4]
    v = struct.unpack('h', left)[0]
    y[i] = v
    if y[i] >=500 or y[i]<=-500:
        normal_sound+=1
    else:
        too_low_sound+=1
    if normal_sound >= int(numframes*0.3):
        print('音量正常', normal_sound, too_low_sound)
    else:
        print('音量太小', normal_sound, too_low_sound)
        response_dict={
            "response":"Your voice is too low"
        }
        response = json.dumps(response_dict)
        return response

In [ ]:
level1=0
level2=0
level3=0
level4=0
level5=0

def check_your_audio():
    # 读取wav文件
    filename = "./audio.wav"
    wavefile = wave.open(filename, 'r')  # open for writing
    # 读取wav文件的四种信息的函数。numframes表示一共读取了几个frames。
    nchannels = wavefile.getnchannels()
    sample_width = wavefile.getsampwidth()
    framerate = wavefile.getframerate()
    numframes = wavefile.getnframes()

    y = np.zeros(numframes)
    return numframes, y, wavefile
            
def load_data_test(data_path):
    with open(data_path, "r",encoding="utf-8") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    X = X[..., np.newaxis]
    X = X[:,np.newaxis]
    
    return X, y

def score_(point):
    global level1, level2, level3, level4, level5
    if point >=0 and point <=20:
        level1+=1
        return ('你的美式口音還需要多加強')
    elif point >=21 and point <=40:
        level2+=1
        return ('你的美式口音有抓到一點訣竅，但還有很多細節沒有到位，多多練習會更好哦')
    elif point >=41 and point <=60:
        level3+=1
        return ('你的美式口音表現很不錯, 但一些口音細節還需要加強')
    elif point >=61 and point <=80:
        level4+=1
        return ('你的美式口音整理表現得很好、細節也有到位, 多練習就完美囉 ')
    elif point >=81 and point <=100:
        level5+=1
        return ('你的美式口英非常標準, 恭喜你')

def prediction(X, model_load):
    score_sum=0
    name_score=0
    point_list=[]
    for i, pre in enumerate(X):
        prediction = model_load.predict(pre)
        # print("prediction: ",prediction)
        predictied_index = np.argmax(prediction, axis=1)
        if int(predictied_index[0]) == 0:
            predictied_index = np.argmin(prediction, axis=1)
        score = (prediction[0][predictied_index])
        # # print(i, predictied_index)
#         print("prediction: ", prediction)
        score_sum += score
        if (i+1)%4==0:
            print((score_sum/4)*100)
            response = int((score_sum/4)*100)
            point_list.append(response)
            score_sum=0
            name_score+=1
    return score_(response)

# PATH
def get_mfcc():
    data = {
    "name":[],
    "labels": [],
    "mfcc": []
    }
    JSON_PATH = "./audio.json"
    SAMPLE_RATE = 22050
    TRACK_DURATION = 10  # measured in seconds   ###### 10 、16秒兩個版本 ######
    SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION

    # PARAMS
    num_mfcc=13
    n_fft=2048
    hop_length=512
    num_segments=4
    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

    # user files transform
    file = "./audio.wav"

    signal, sample_rate = librosa.load(file, sr=SAMPLE_RATE)

    # process all segments of audio file
    for id, d in enumerate(range(num_segments)):
        # calculate start and finish sample for current segment
        start = samples_per_segment * d
        finish = start + samples_per_segment

        # extract mfcc
        mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
                                    hop_length=hop_length)
        mfcc = mfcc.T
        mfcc_cut = mfcc.copy()
        mfcc_cut[abs(mfcc_cut)<10]=0
        # store only mfcc feature with expected number of vectors
        if len(mfcc_cut) == num_mfcc_vectors_per_segment:
            # print(mfcc)
            name = file.split("./")[1].split(".wav")[0]
            if (id+1)%4 == 0:
                data["name"].append(name)
            data["mfcc"].append(mfcc.tolist())
            data["labels"].append(0)

    with open("audio.json", "w",encoding="utf-8") as fp:
        json.dump(data, fp, indent=4)

    return "MFCC OK"

def back_img():
    img_path="./audio.png"
    with open(img_path, 'rb') as img_f:
        img_stream = img_f.read()
        img_stream = base64.b64encode(img_stream)
    return img_stream

def plot_picture():
    plt.plot(point_list, color="b", linewidth=3)
    plt.figsize=(12,8)
    plt.title('Mr. XXX Growth Record',fontsize=20)
    plt.xlabel('date',fontsize=12)
    plt.ylabel('point',fontsize=12)
    plt.xtick(fontsize=10)
    plt.ytick(fontsize=10)
    plt.grid(True)
    plt.show()

In [ ]:
app = Flask(__name__)
@app.route('/file', methods=['GET', "POST"])
@cross_origin()
def files():
    normal_sound=0
    too_low_sound=0
    file = request.files['file']
    file.save('./audio.wav')
    ##### 11/30新增
    teacher = get_wave_word(file_path)
    student = get_wave_word('audio.wav')
    point = model.wv.similarity(teacher, student)
    print("你的語意分數是: ", point)
    if point <= 0.8:
        return "妳唸得不太正確, 請重新再唸一次!!"
    #####
    check_your_voice_Volume()
    numframes, y, wavefile = check_your_audio()
    for index, i in enumerate(range(numframes)):
        val = wavefile.readframes(1)
        left = val[0:2]
        # right = val[2:4]
        v = struct.unpack('h', left)[0]
        y[i] = v
        if y[i] >=500 or y[i]<=-500:
            normal_sound+=1
        else:
            too_low_sound+=1
    if normal_sound >= int(numframes*0.3):
        print('音量正常', normal_sound, too_low_sound)
        response_dict={
            "response":"Voice OK"
        }
        response = json.dumps(response_dict)
        return response
    else:
        print('音量太小', normal_sound, too_low_sound)
        response_dict={
            "response":"Your voice is too low"
        }
        response = json.dumps(response_dict)
        return response
    get_mfcc()
    TEST_PATH = "./audio.json"
    X, y = load_data_test(TEST_PATH)
    # 16s "model_version_v7.h5"
    model_load = load_model("model/New 0.341 - 0.9212.h5")
    response = prediction(X, model_load)
    print(response)
    # picture
    oscillogram_spectrum()
    Bargraph()
    picture = back_img()
    re = {
        "response" : response,
        "picture" : picture.decode('utf-8')
    }
    re = json.dumps(re)
    return re

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0',port = 5003)

In [ ]:
# 读取wav文件
filename = "./audio.wav"
wavefile = wave.open(filename, 'r')  # open for writing
# 读取wav文件的四种信息的函数。numframes表示一共读取了几个frames。
nchannels = wavefile.getnchannels()
sample_width = wavefile.getsampwidth()
framerate = wavefile.getframerate()
numframes = wavefile.getnframes()

y = np.zeros(numframes)

print(nchannels)
print(sample_width)
print(framerate)
print(numframes)

s = numframes / framerate

In [ ]:
# 读取wav文件
filename = "./audio.wav"
y, sr = librosa.load(filename, sr=None)
print(y, sr)

In [ ]:
feature = librosa.feature.melspectrogram(y, sr, n_fft=1024, hop_length=512, n_mels=128)
logmelspc = librosa.power_to_db(feature)
logmelspc.shape

In [ ]:
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
mfccs.shape

In [ ]:
import speech_recognition as sr
r = sr.Recognizer()                        #預設辨識英文
with sr.WavFile("C:/Users/PC06/Downloads/lili colins/1/Lily Collins Discusses Filming Tolkien and Ted Bundy Biopic This Morning_1.wav") as source:  #讀取wav檔
    audio = r.record(source)
try:
    print("Transcription: " + r.recognize_google(audio,language="en"))
                                          #使用Google的服務
except LookupError:
    print("Could not understand audio")

In [ ]:
def get_wave_word():
    r = sr.Recognizer()                        #預設辨識英文
    with sr.WavFile("./voice/Will Smith 7.wav") as source:  #讀取wav檔
        audio = r.record(source)
    try:
        print("Transcription: " , r.recognize_google(audio,language="en"))
                                              #使用Google的服務
    except:
         print("請再唸一次")

In [ ]:
get_wave_word()